In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1
import cv2
import numpy as np

# ==========================================
# 1. NASTAVENÍ A INICIALIZACE
# ==========================================

DATASET_DIR = 'dataset_xy'
try:
    if not os.path.exists(DATASET_DIR):
        os.makedirs(DATASET_DIR)
except:
    print("Nelze vytvořit složku dataset_xy")

# Inicializace kamery
try:
    # Zkusíme získat instanci, pokud už běží, jinak vytvoříme novou
    camera = Camera.instance(width=224, height=224)
except Exception as e:
    print(f"Chyba kamery: {e}")

# Pokus o import ipyevents
try:
    from ipyevents import Event
    IPYEVENTS_AVAILABLE = True
except ImportError:
    print("Chybí 'ipyevents'. Nainstalujte v terminálu: sudo pip3 install ipyevents")
    IPYEVENTS_AVAILABLE = False

# ==========================================
# 2. DEFINICE WIDGETŮ
# ==========================================

# Živé video (vlevo)
image_widget = widgets.Image(format='jpeg', width=224, height=224)

# Náhled s terčíkem (vpravo)
empty_image = np.zeros((224, 224, 3), dtype=np.uint8)
target_widget = widgets.Image(value=bgr8_to_jpeg(empty_image), format='jpeg', width=224, height=224)

# Posuvníky
x_slider = widgets.FloatSlider(min=0, max=224, step=1, value=112, description='x')
y_slider = widgets.FloatSlider(min=0, max=224, step=1, value=224, description='y')

# Tlačítko Uložit
save_button = widgets.Button(description='Uložit Snímek', button_style='success')

# Počítadlo
count_widget = widgets.IntText(description='Počet:', value=len(os.listdir(DATASET_DIR)))

# ==========================================
# 3. FUNKCE LOGIKY
# ==========================================

def display_target(change):
    """
    Kreslí křížek do pravého obrázku.
    """
    # Pokud kamera ještě nemá data, nebo je vypnutá, použijeme černý obraz
    if camera.value is None:
        return
        
    # Vytvoříme kopii aktuálního snímku z numpy pole kamery
    image = np.copy(camera.value)
    
    # Souřadnice ze sliderů
    x = int(x_slider.value)
    y = int(y_slider.value)
    
    # Vykreslení zeleného křížku
    # Ošetření okrajů, aby cv2 nespadlo, když jsme úplně na kraji
    x = max(0, min(223, x))
    y = max(0, min(223, y))
    
    cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    cv2.line(image, (x - 10, y), (x + 10, y), (0, 255, 0), 2)
    cv2.line(image, (x, y - 10), (x, y + 10), (0, 255, 0), 2)
    
    # Aktualizace pravého widgetu
    target_widget.value = bgr8_to_jpeg(image)

def save_snapshot(b):
    """Uloží snímek a souřadnice."""
    x = x_slider.value
    y = y_slider.value
    uuid = uuid1()
    
    # Formát názvu: xy_<X>_<Y>_<UUID>.jpg
    image_path = os.path.join(DATASET_DIR, f"xy_{int(x):03d}_{int(y):03d}_{uuid}.jpg")
    
    if camera.value is not None:
        with open(image_path, 'wb') as f:
            f.write(bgr8_to_jpeg(camera.value))
        
        # Aktualizace počítadla
        count_widget.value = len(os.listdir(DATASET_DIR))
        
        # (Volitelné) Efekt bliknutí tlačítka
        save_button.description = "ULOŽENO!"
        save_button.button_style = "info"
        import time
        # Toto v Jupyteru nefunguje ideálně bez async, ale pro info stačí
        # Button resetujeme při dalším kliknutí

def handle_image_click(event):
    """Reaguje na kliknutí do živého videa."""
    # OPRAVA: Musíme zkontrolovat, zda event obsahuje souřadnice
    if 'relativeX' in event and 'relativeY' in event:
        x = event['relativeX']
        y = event['relativeY'] # ZDE BYLA CHYBA (y = event)
        
        # Ošetření mezí
        x = max(0, min(224, x))
        y = max(0, min(224, y))
        
        # Aktualizace sliderů (to automaticky spustí display_target)
        x_slider.value = x
        y_slider.value = y
        
        # Pokud chcete automaticky ukládat při kliknutí, odkomentujte řádek níže:
        # save_snapshot(None)

# ==========================================
# 4. PROPOJENÍ A LAYOUT (OPRAVENO)
# ==========================================

# 1. Propojení kamery s levým widgetem (live stream)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# 2. Propojení sliderů s funkcí kreslení terčíku
x_slider.observe(display_target, names='value')
y_slider.observe(display_target, names='value')

# 3. Tlačítko Save
save_button.on_click(save_snapshot)

# 4. Klikání do obrazu
if IPYEVENTS_AVAILABLE:
    interaction = Event(source=image_widget, watched_events=['click'])
    interaction.on_dom_event(handle_image_click)

# 5. Sestavení Layoutu - OPRAVENA SYNTAXE
# Vytvoříme řádek s obrázky
images_row = widgets.HBox([image_widget, target_widget])
# Vytvoříme řádek s ovládáním
controls_row = widgets.HBox([count_widget, save_button])

# Celkový layout sloupce
layout = widgets.VBox([
    images_row, 
    x_slider, 
    y_slider, 
    controls_row
])

display(layout)

# 6. Inicializační "šťouch", aby se zobrazil pravý obrázek hned
# (Pokud kamera běží, zavoláme funkci ručně)
if camera.value is not None:
    display_target(None)

In [2]:
DATASET_DIR = 'dataset_xy'

try:
    os.makedirs(DATASET_DIR)
    print(f"Složka '{DATASET_DIR}' byla vytvořena.")
except FileExistsError:
    print(f"Složka '{DATASET_DIR}' již existuje.")

Složka 'dataset_xy' již existuje.


In [3]:
import cv2
import numpy as np

def save_snapshot(x, y):
    """Uloží snímek s názvem xy_<x>_<y>_<uuid>.jpg"""
    uuid = uuid1()
    image_path = os.path.join(DATASET_DIR, f"xy_{x:.2f}_{y:.2f}_{uuid}.jpg")
    
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
        
    # print(f"Uloženo: {image_path}") # Odkomentujte pro výpis

def display_target(change):
    """
    Tato funkce vykreslí na vedlejší obrazovku 'terčík'.
    OPRAVA: Čteme hodnoty přímo ze sliderů, ne z 'change'.
    """
    # Vytvoření prázdného (černého) obrázku
    image = np.zeros((224, 224, 3), dtype=np.uint8)
    
    # Přečteme aktuální hodnoty přímo z widgetů
    current_x = x_slider.value
    current_y = y_slider.value
    
    # Přepočet z <-1, 1> zpět na pixely <0, 224>
    pixel_x = int(224 * (current_x + 1.0) / 2.0)
    pixel_y = int(224 * (current_y + 1.0) / 2.0)
    
    # Ošetření okrajů (aby křížek nevytekl z obrazu)
    pixel_x = max(0, min(224, pixel_x))
    pixel_y = max(0, min(224, pixel_y))
    
    # Vykreslení zeleného křížku
    cv2.circle(image, (pixel_x, pixel_y), 8, (0, 255, 0), 3)
    cv2.line(image, (pixel_x - 10, pixel_y), (pixel_x + 10, pixel_y), (0, 255, 0), 2)
    cv2.line(image, (pixel_x, pixel_y - 10), (pixel_x, pixel_y + 10), (0, 255, 0), 2)
    
    target_widget.value = bgr8_to_jpeg(image)

# Sledování změn sliderů
x_slider.observe(display_target, names='value')
y_slider.observe(display_target, names='value')

# Funkce pro zpracování kliknutí do obrázku
def handle_image_click(x, y):
    # Widget vrací souřadnice v pixelech (např. 112, 50)
    
    # Normalizace X (z 0..224 na -1..1)
    norm_x = 2.0 * (x / 224.0) - 1.0
    
    # Normalizace Y (z 0..224 na -1..1)
    norm_y = 2.0 * (y / 224.0) - 1.0
    
    # Aktualizace sliderů (toto automaticky spustí display_target díky .observe)
    x_slider.value = norm_x
    y_slider.value = norm_y
    
    # ULOŽENÍ SNÍMKU
    save_snapshot(norm_x, norm_y)

# Aktivace ipyevents pro klikání
try:
    from ipyevents import Event
    
    # Sledujeme widget s obrazem z kamery
    interaction = Event(source=image_widget, watched_events=['click'])
    
    def on_click(event):
        handle_image_click(event['eventX'], event['eventY'])
        
    interaction.on_dom_event(on_click)
    print("Klikání myší aktivováno! Klikni do obrazu pro uložení.")
    
except ImportError:
    print("Knihovna 'ipyevents' chybí. Musíš nastavovat X a Y pomocí sliderů a volat save_snapshot ručně.")

# Inicializace terčíku na střed (aby nebyl broken image)
display_target(None)

Klikání myší aktivováno! Klikni do obrazu pro uložení.


In [4]:
count = len(os.listdir(DATASET_DIR))
print(f"Aktuální počet trénovacích snímků pro regresi: {count}")

# Tlačítko pro zabalení dat
# !zip -r -q dataset_xy.zip dataset_xy

In [1]:
from ipyevents import Event
import ipywidgets as widgets
from IPython.display import display

l = widgets.Label('Klikni na modrou plochu níže:')
out = widgets.Output()
# Vytvoříme testovací widget
box = widgets.HTML('<div style="width: 200px; height: 200px; background-color: blue;"></div>')

# Připojíme událost
d = Event(source=box, watched_events=['click'])

def handle_event(event):
    with out:
        print(f"Kliknuto! X: {event['relativeX']}, Y: {event['relativeY']}")

d.on_dom_event(handle_event)

display(l, box, out)

Label(value='Klikni na modrou plochu níže:')

HTML(value='<div style="width: 200px; height: 200px; background-color: blue;"></div>')

Output()